# Programación declarativa @ URJC
# Programación funcional
## Examen Convocatoria Extraordinaria (2 de julio de 2021)
## Curso 2020-2021

# Definiciones auxiliares

In [ ]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

### Algunas funciones sobre tipos estándar de la librería de Scala

In [ ]:
object Signatures{
    abstract class List[A]{
        
        // Common HOFs
        def foldRight[B](directSol: B)(composeSol: (A, B) => B): B
        def foldLeft[B](initial: B)(update: (B, A) => B): B
        def map[B](f: A => B): List[B]
        def flatMap[B](f: A => List[B]): List[B]
        def filter(f: A => Boolean): List[A]
        def forall(pred: A => Boolean): Boolean
        def exists(pred: A => Boolean): Boolean
        
        // Reverse a list
        // e.g. List(1,2,3).reverse==List(3,2,1)
        def reverse: List[A]
        
        // Take the first `n` elements of the list
        // e.g. List(1,2,3).take(2) == List(1,2)
        //      List(1,2,3).take(0) == List()
        //      List(1,2,3).take(5) == List(1,2,3)
        def take(n: Int): List[A]
        
        // Drop the first `n` elements of the list 
        // e.g. List(1,2,3).drop(2) == List(3)
        //      List(1,2,3).drop(0) == List(1,2,3)
        //      List(1,2,3).drop(4) == List()
        def drop(n: Int): List[A]

        // List concatenation
        // e.g. List(1,2,3).concat(List(4,5)) == List(1,2,3,4,5)
        def concat(l: List[A]): List[A]
    }
    
    abstract class Option[A]{
        // Test whether the value is defined (i.e. `Some`) or not (i.e. `None`)
        def isDefined: Boolean 
        def map[B](f: A => B): Option[B]
    }
    
    abstract class Either[A, B]{
        // Test whether the value is left or right
        def isLeft: Boolean 
        def isRight: Boolean 
        def map[C](f: B => C): Either[A, C]
    }
}

### Definiciones auxiliares sobre la correspondencia Curry-Howard

In [ ]:
type Not[P] = P => Nothing
type <=>[P, Q] = (P => Q, Q => P)
type Or[P, Q] = Either[P, Q]
type And[P, Q] = (P, Q)

# Ejercicio 1


__a) (2 puntos)__ Utiliza la correspondencia de Curry-Howard para demostrar que las siguientes proposiciones representan teoremas de la lógica intuicionista: 

$\vdash \neg (p \vee q) \rightarrow \neg p \wedge \neg q$


In [ ]:
def proof[P, Q](p1: Not[Either[P, Q]]): (Not[P], Not[Q]) = 
    ((p:P) => p1(Left(p)), 
     (q: Q) => p1(Right(q)))

$\vdash \neg p \wedge \neg q \rightarrow \neg (p \vee q)$

In [ ]:
def proof[P, Q](p1: (Not[P], Not[Q])): Not[Either[P, Q]]  = 
    { 
        case Left(p) => p1._1(p)
        case Right(q) => p1._2(q)
    }

__b) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar el siguiente teorema de la lógica clásica: 

$\vdash \neg(\neg p \vee \neg q) \rightarrow p \wedge q$

Supóngase para ello que la ley del tercio excluso se cumple para las variables proposicionales $p$ y $q$, es decir, que las fórmulas $p ∨ ¬p$ y $q ∨ ¬q$  pueden utilizarse como premisas.


In [ ]:
def proof[P, Q](pnp: Either[P, Not[P]], qnq: Either[Q, Not[Q]]): Not[Either[Not[P], Not[Q]]] => (P, Q) = 
    (f: Either[Not[P], Not[Q]] => Nothing) => 
        (pnp, qnq) match {
            case (Left(p), Left(q)) => (p, q)
            case (Left(p), Right(nq)) => (p, f(Right(nq)))
            case (Right(np), Left(q)) => (f(Left(np)), q)
            case (Right(np), Right(nq)) => (f(Left(np)), f(Right(nq)))
        }

# Ejercicio 2 
__(2 puntos)__

Demuestra el isomorfismo entre tipos algebraicos de datos correspondiente a la siguiente igualdad algebraica: 

<p style="text-align: center;">$(y*z)^x = y^x*z^x$, para todo $x$, $y$, $z \in \mathbb{N}$</p>

In [ ]:
def from[X, Y, Z](f: X => (Y, Z)): (X => Y, X => Z) = 
    (x => f(x)._1, x => f(x)._2)


In [ ]:
def to[X, Y, Z](t: (X => Y, X => Z)): X => (Y, Z)  = 
    x => (t._1(x), t._2(x))


# Ejercicio 3 (variante A)
__(5 puntos)__

La función `split` recibe una lista de pares y devuelve un par de listas formadas por los elementos de la primera y segunda componentes de cada par, respectivamente. El comportamiento de la función se ilustra en el siguiente test unitario:


In [ ]:
class SplitTest_A(
    split: List[(Int, Boolean)] => (List[Int], List[Boolean])
) extends FlatSpec with Matchers{
    "split" should "work" in {
        split(List()) shouldBe (List(), List())
        split(List((1, true), (2, false), (3, true))) shouldBe (List(1,2,3), List(true, false, true))
        split(List((0, true), (0, true))) shouldBe (List(0,0), List(true, true))
    }
}

__a) (1 punto)__ Implementa la función `split` de manera recursiva (no final).

In [ ]:
def splitR_A[A, B](l: List[(A, B)]): (List[A], List[B]) = 
    l match {
        case Nil => (Nil, Nil)
        case (a,b) :: tail => 
            val (la, lb) = splitR_A(tail)
            (a :: la, b :: lb)
    }

In [ ]:
run(new SplitTest_A(splitR_A))

__b) (1 punto)__ Implementa la función `split` utilizando __`foldRight`__.

In [ ]:
def splitFR_A[A, B](l: List[(A, B)]): (List[A], List[B]) = 
    l.foldRight((List[A](), List[B]())){
        case ((a,b), (la, lb)) => 
            (a :: la, b :: lb)
    }

In [ ]:
run(new SplitTest_A(splitFR_A))

__c) (1 punto)__ Implementa la función `split` mediante recursión por cola (o final):

In [ ]:
def splitTR_A[A, B](l: List[(A, B)]): (List[A], List[B]) = {
    def aux(acc: (List[A], List[B]), l: List[(A, B)]): (List[A], List[B]) = 
        (acc, l) match {
            case (_, Nil) => 
                acc
            case ((la, lb), (a, b) :: tail) => 
                aux((a :: la, b :: lb), tail)
        }
    
    aux((Nil, Nil), l) match {
        case (la, lb) => (la.reverse, lb.reverse)
    }
}

In [ ]:
run(new SplitTest_A(splitTR_A))

__d) (1 punto)__ Implementa la función `split` utilizando __`foldLeft`__.

In [ ]:
def splitFL_A[A, B](l: List[(A, B)]): (List[A], List[B]) = 
    l.foldLeft((List[A](), List[B]())){
        case ((la, lb), (a,b)) => 
            (a :: la, b :: lb)
    } match {
        case (la, lb) => (la.reverse, lb.reverse)
    }

In [ ]:
run(new SplitTest_A(splitFL_A))

__e) (1 punto)__ Dada una lista de puntos del plano representados mediante coordenadas polares, se desean obtener los valores de las abscisas y ordenadas de sus coordenadas rectangulares. Concretamente, dados los tipos de datos:

In [ ]:
case class Polar(radius: Double, angle: Double)
type Rectangular = (Double, Double)

y la función de conversión entre coordenadas polares y rectangulares: 

In [ ]:
import scala.math.{Pi, sin, cos}

def from(p: Polar): Rectangular = 
    (p.radius * cos(p.angle), p.radius * sin(p.angle))

se desea implementar una función `components` que satisfaga el siguiente test unitario:

In [ ]:
class ComponentsTest(
    components: List[Polar] => (List[Double], List[Double])
) extends FlatSpec with Matchers{
    "components" should "work" in {
        components(List(Polar(1,0), Polar(2,Pi/4), Polar(3,Pi/2), Polar(4,Pi))) shouldBe 
            (List(cos(0), 2*cos(Pi/4), 3*cos(Pi/2), 4*cos(Pi)), 
            List(sin(0), 2*sin(Pi/4), 3*sin(Pi/2), 4*sin(Pi)))
    }
}

Implementa la función `components` utilizando la función `split` y alguna de las funciones de orden superior explicadas en clase (`map`, `flatMap`, `foldLeft`, `foldRight`, etc.), de tal forma que la implementación sea lo más __concisa__ posible.

In [ ]:
def components(l: List[Polar]): (List[Double], List[Double]) = 
    splitR_A(l.map(from))

In [ ]:
run(new ComponentsTest(components))

# Ejercicio 3 (variante B)
__(5 puntos)__

La función `split` recibe una lista de valores de tipo `Either`, y devuelve un par de listas formadas por los elementos de tipo `Left` y `Right`, respectivamente. El comportamiento de la función se ilustra en el siguiente test unitario:


In [ ]:
class SplitTest_B(
    split: List[Either[Int, Boolean]] => (List[Int], List[Boolean])
) extends FlatSpec with Matchers{
    "split" should "work" in {
        split(List()) shouldBe (List(), List())
        split(List(Left(1), Right(false), Left(3))) shouldBe (List(1,3), List(false))
        split(List(Right(true), Right(true))) shouldBe (List(), List(true, true))
    }
}

__a) (1 punto)__ Implementa la función `split` de manera recursiva (no final).

In [ ]:
def splitR_B[A, B](l: List[Either[A, B]]): (List[A], List[B]) = 
    l match {
        case Nil => (Nil, Nil)
        case head :: tail => 
            (head, splitR_B(tail)) match {
                case (Left(a), (la, lb)) => 
                    (a :: la, lb)
                case (Right(b), (la, lb)) => 
                    (la, b :: lb)
            }
    }

In [ ]:
run(new SplitTest_B(splitR_B))

__b) (1 punto)__ Implementa la función `split` utilizando __`foldRight`__.

In [ ]:
def splitFR_B[A, B](l: List[Either[A, B]]): (List[A], List[B]) = 
    l.foldRight((List[A](), List[B]())){
        case (Left(a), (la, lb)) => 
            (a :: la, lb)
        case (Right(b), (la, lb)) => 
            (la, b :: lb)
    }

In [ ]:
run(new SplitTest_B(splitFR_B))

__c) (1 punto)__ Implementa la función `split` mediante recursión por cola (o final):

In [ ]:
def splitTR_B[A, B](l: List[Either[A, B]]): (List[A], List[B]) = {
    def aux(acc: (List[A], List[B]), l: List[Either[A, B]]): (List[A], List[B]) = 
        (acc, l) match {
            case (_, Nil) => 
                acc
            case ((la, lb), Left(a) :: tail) => 
                aux((a :: la, lb), tail)
            case ((la, lb), Right(b) :: tail) => 
                aux((la, b :: lb), tail)
        }
    
    aux((Nil, Nil), l) match {
        case (la, lb) => (la.reverse, lb.reverse)
    }
}

In [ ]:
run(new SplitTest_B(splitTR_B))

__d) (1 punto)__ Implementa la función `split` utilizando __`foldLeft`__.

In [ ]:
def splitFL_B[A, B](l: List[Either[A, B]]): (List[A], List[B]) = 
    l.foldLeft((List[A](), List[B]())){
        case ((la, lb), Left(a)) => 
            (a :: la, lb)
        case ((la, lb), Right(b)) => 
            (la, b :: lb)
    } match {
        case (la, lb) => (la.reverse, lb.reverse)
    }

In [ ]:
run(new SplitTest_B(splitFL_B))

__e) (1 punto)__ Implementa la función `split` utilizando la función de orden superior `filter`:

In [ ]:
def splitF[A, B](l: List[Either[A, B]]): (List[A], List[B]) = 
    (l.filter(_.isLeft).map{ case Left(a) => a}, 
     l.filter(_.isRight).map{ case Right(b) => b})

In [ ]:
run(new SplitTest_B(splitF))